# Week3

- Sequence to Sequence Model
- Beam Search
- Attention

## Seq2Seq

seq2seq 是一个Encoder–Decoder 结构的网络，它的输入是一个序列，输出也是一个序列， Encoder 中将一个可变长度的信号序列变为固定长度的向量表达，Decoder 将这个固定长度的向量变成可变长度的目标的信号序列。

<img src="src/pictures/c5_w3_2.jpeg">

RNN中当前时间步的隐藏状态由上一个时间步的隐藏状态和输入决定
$$h_t = f(h_{t-1}, x_t)$$
获得各个时间步的隐藏层, 将隐藏层信息进行汇总生成语义向量
$$C=q(h_1, h_2, h_3,...,h_{T_x})$$

最简单的语义向量就是$T_x$时间步的隐藏层作为语义向量.

解码阶段, 根据语义C和之前已经生成的输出序列来预测下一时间步的输出单词$y_t$, 有
$$y_t=argmax_y P(y_t) \\ 
\qquad = argmax \prod_{t=1}^T p(y_t|y_1, y_2, \dots, y_{t-1}, C) \\
\qquad = argmax \sum_{t=1}^T log p(y_t|y_1, y_2, \dots, y_{t-1}, C)$$

对于机器翻译等问题, 输出的目标是获得最可能的句子, 即要求$P(y_1, y_2, \dots, y_t|X)$整体最大, 如果使用Greedy Search, 在t时间步选择概率最大的单词$y_t$作为输出, 不能够得到整体最优, 因此提出了Beam Search(束搜索)来获得近似全局最优的句子.

## Beam Search

束搜索的直观理解是在t时间步选择概率最大的B个$\{y_1, y_2, \dots, y_t\}$序列代替只选择概率当前概率最大的序列, 其中B称为束宽度. 如B=2的情形如下图:

![B=2](src/pictures/c5_w3_4.png)

基本的B结构偏向于短句, 因为连乘小于1的数值, 越长数值一定越小, 为解决这个问题, 提出长度正则化, 将序列的概率计算变为:
$$\frac{1}{T^\alpha} argmax \sum_{t=1}^T log p(y_t|y_1, y_2, \dots, y_{t-1}, C)$$

其中$\alpha$控制正则化的柔和程度, 当$\alpha=1$时最严苛, 当$\alpha = 0$时没有正则化.

B的参数选择也对搜索的性能具有影响, B越小,搜索越快搜索结果更差;B越大计算量更大搜索结果更好. 工业上常采用B=10, 20, 研究中最大常才用到1000.

### 加入BS的误差分析

如何判断使用BS机制和RNN结构进行训练时, 出现误差时是哪个部分出现了问题?

可以对输入的目标结果(人工翻译的结果), 使用上述的RNN的权重和上述的评价式计算其概率$y^*$, 同时计算机器翻译的结果$\hat{y}$, 若$y^* > \hat{y}$, 说明BS不能够搜索到最优的结果;若$y^* \leq \hat{y}$, 说明RNN不能正确的估算正确结果的概率.

## Attention

Attention机制使神经网路偶在当前时间步只关注序列的一部分.
